# Draft of script to merge network and physical data

This script has some code that is useful to understand dissecting packets with scapy and merging network and physical data

In [1]:
import cip # library that has the packet structure of minicps network data
from scapy.all import * # scapy python is a tool to disect network packets
from scapy.contrib import modbus
import pandas as pd 
from decimal import Decimal
import time

In [9]:
data = []
network_data = rdpcap("../../Jupyter_notebooks/experiment_data/way_issue/plc2-eth0.pcap")
physical_data = pd.read_csv("../../Jupyter_notebooks/experiment_data/way_issue/physical_process.csv")
# This for goes over all the physical data. The physical data has 672 iterations. What we try to do is relate one physical entry 
# into the .csv file with all the packets involved in that data entry
for i in range(len(physical_data)):
    current_epoch = (datetime.strptime(physical_data.get('timestamps')[i], "%M:%S.%f")-datetime(1970, 1, 1)).total_seconds()
    previous_epoch = (datetime.strptime(physical_data.get('timestamps')[i], "%M:%S.%f")-datetime(1970, 1, 1)).total_seconds()    
    for nwk in network_data:
        if (Decimal(nwk.time)+28800) >= previous_epoch and (Decimal(nwk.time)+28800) <= current_epoch:  
            data_frag = {}
            data_frag['network_data'] = nwk
            data_frag['T_LVL'] = physical_data.get('TANK_LEVEL')[i]   
            data_frag['RESERVOIR_LEVEL'] = physical_data.get('RESERVOIR_LEVEL')[i]
            data_frag['J421'] = physical_data.get('J421')[i]
            data_frag['J332'] = physical_data.get('J332')[i]
            data_frag['J156'] = physical_data.get('J156')[i]
            data_frag['J39'] = physical_data.get('J39')[i]
            data_frag['J269'] = physical_data.get('J269')[i]
            data_frag['J273'] = physical_data.get('J273')[i]
            data_frag['J280'] = physical_data.get('J280')[i]
            data_frag['J285'] = physical_data.get('J285')[i]
            data_frag['FLOW_PUMP1'] = physical_data.get('FLOW_PUMP1')[i]
            data_frag['FLOW_PUMP2'] = physical_data.get('FLOW_PUMP2')[i]
            data_frag['STATUS_PUMP1'] = physical_data.get('STATUS_PUMP1')[i]
            data_frag['STATUS_PUMP2'] = physical_data.get('STATUS_PUMP2')[i]
            data_frag['Attack#01'] = physical_data.get('Attack#01')[i]
            data_frag['Attack#02'] = physical_data.get('Attack#02')[i]
            
            data.append(data_frag)

In [18]:
len(data[0]['network_data'])

74

In [12]:
data[0]['network_data']

<Ether  dst=b6:af:26:39:cd:64 src=5a:87:33:69:f9:53 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=60 id=42948 flags=DF frag=0 ttl=64 proto=tcp chksum=0xf89 src=192.168.1.20 dst=192.168.1.10 |<TCP  sport=57056 dport=44818 seq=3682037379 ack=0 dataofs=10 reserved=0 flags=S window=29200 chksum=0x839d urgptr=0 options=[('MSS', 1460), ('SAckOK', b''), ('Timestamp', (26727893, 0)), ('NOP', None), ('WScale', 9)] |>>>

In [27]:
data[56]

{'network_data': <Ether  dst=5a:87:33:69:f9:53 src=b6:af:26:39:cd:64 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=102 id=22635 flags=DF frag=0 ttl=64 proto=tcp chksum=0x5eb8 src=192.168.1.10 dst=192.168.1.20 |<TCP  sport=44818 dport=57088 seq=528424531 ack=1752646756 dataofs=8 reserved=0 flags=PA window=57 chksum=0x83c7 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (26728370, 26728366))] |<ENIP_TCP  command_id=SendRRData length=26 session=3557626027 status=success sender_context=48 options=0 |<ENIP_SendRRData  interface_handle=0 timeout=0 count=2 items=[<ENIP_SendUnitData_Item  type_id=null_address length=0 |>, <ENIP_SendUnitData_Item  type_id=unconn_message length=10 |<CIP  direction=response service=Read_Tag_Service status=[<CIP_ResponseStatus  status=Success>] |<Raw  load='\xca\x00\xe1A8@' |>>>] |>>>>>,
 'T_LVL': 2.8064021050558665,
 'RESERVOIR_LEVEL': 59.0,
 'J421': 37.62283634538997,
 'J332': 22.28028676247913,
 'J156': 35.35082348403,
 'J39': 28.32070322035819,
 'J

In [10]:
data[56]['network_data'].show()

###[ Ethernet ]### 
  dst       = 5a:87:33:69:f9:53
  src       = b6:af:26:39:cd:64
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 102
     id        = 22635
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x5eb8
     src       = 192.168.1.10
     dst       = 192.168.1.20
     \options   \
###[ TCP ]### 
        sport     = 44818
        dport     = 57088
        seq       = 528424531
        ack       = 1752646756
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 57
        chksum    = 0x83c7
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (26728370, 26728366))]
###[ ENIP_TCP ]### 
           command_id= SendRRData
           length    = 26
           session   = 3557626027
           status    = success
           sender_context= 48
           options   = 0
###[ ENIP_SendRRData ]### 
              in

In [28]:
data[56]['network_data']['IP']['TCP']['ENIP_TCP']['CIP']

<CIP  direction=response service=Read_Tag_Service status=[<CIP_ResponseStatus  status=Success>] |<Raw  load='\xca\x00\xe1A8@' |>>